# 章節 15：維度處理

## 15.8 我們還沒搞定

一次更新一個物種解決了一個問題，但卻又衍生了另一個。每一次更新的計算時間與 k\*m 成正比，k 是有多少假設且 m 是有多少觀察到的物種。所以如果我們做 m 次的更新，總共的計算時間與 k\*m^2 成正比。

我們同樣可以用 15.6 小節的技巧來加速：我們捨棄 Dirichlet 的物件陣列並且將兩層的結構折疊成一層。以下是改良的版本。

<pre>
class Species5(Species2):
    def Update(self, data):
        m = len(data)
        for i in range(m):
            self.UpdateOne(i+1, data[i])
            self.params[i] += data[i]
</pre>

Species5 繼承自 Species2，所以 Species5 物件有：
- ns: 種類數量的分佈
- probs: Dirichlet 分佈的參數

Update 分法跟之前小節的很像，依次對觀察到的物種更新，我們稱做 UpdateOne 方法：

<pre>
# class Species5
    def UpdateOne(self, i, count):
        likes = numpy.zeros(len(self.ns), dtype=numpy.double)
        for i in range(self.iterations):
            likes += self.SampleLikelihood(i, count)
        unseen_species = [n-i+1 for n in self.ns]
        likes *= unseen_species
        self.probs *= likes
        self.probs /= self.probs.sum()
</pre>

這方法跟 Species2.Update 很類似，但有兩個改變：
- UpdateOne 參數不同。傳入用第 i 個物種出現的次數 count，而非整個觀察到的資料。
- 這邊依樣有校正因子，依據未觀察到的物種數量，就像 Species4.Likelihood。這邊差異在我們用陣列相乘一次調整所有的似然性。

最後是 SampleLikelihood 方法。

<pre>
# class Species5
    def SampleLikelihood(self, i, count):
        gammas = numpy.random.gamma(self.params)
        sums = numpy.cumsum(gammas)[self.ns[0]-1:]
        ps = gammas[i-1] / sums
        log_likes = numpy.log(ps) * count
        log_likes -= numpy.max(log_likes)
        likes = numpy.exp(log_likes)
        return likes
</pre>

和 Species2.SampleLikelihood 類似; 差別在每次只更新一種物種，所以不需要迭代。

Species5 版本的計算時間跟 k 的假設數量成正比。我們需要執行 m 次，所以計算時間與 k*\m 成正比。而且需要得到精確的結果所需要的迭代次數通常很小。